<a href="https://colab.research.google.com/github/camdsDS/Cesar_Portfolio/blob/main/House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
#alvo
respostas = train['SalePrice']

In [4]:
#dropando ID e alvo do treino
train = train.drop(['Id','SalePrice'], axis = 1)
test = test.drop('Id', axis=1)

In [5]:
#Features
train.columns
features = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition']

In [6]:
#a coluna MSSubClass é categórica, porém os dados são números.
#Para ficar claro, vou inserir um 'A' na frente de cada dado
train['MSSubClass'] = [str(x)+'A' for x in train['MSSubClass']]
test['MSSubClass'] = [str(x)+'A' for x in test['MSSubClass']]
#poderia ter feito também por
#train['MSSubClass'] = train['MSSubClass'].apply(lambda x: str(x) +'A')

In [7]:
#função que dropa colunas com uma porcentagem de valores miss superior a um valor P_miss arbitrário
#P_miss é a menor % tolerada de miss
def DropaMiss(df, P_miss=0):
  df_miss2 = ((df.isna().sum()/len(df))*100).to_frame().rename(columns={0:'P_miss'}).reset_index().sort_values('P_miss', ascending = False)
  df_miss = ((df.isna().sum()/len(df))*100).to_frame().rename(columns={0:'P_miss'}).reset_index()
  df_miss = df_miss[df_miss['P_miss'] <= P_miss]
  df = df[df_miss['index'].to_list()]
  return df,df_miss2

In [8]:
#função para dividir o df treino em 2 tipos de dataframe: numéricas e categóricas
def AutoFrames(df):
  tipo = df.dtypes.to_frame().reset_index().rename(columns={"index": "feature", 0: "formato"}) 

  numerica = []
  categorica = []
  
  for x in range(0, len(tipo)):
    if tipo.formato[x] == 'int64' or tipo.formato[x] == 'float64':
      numerica.append(tipo.feature[x])
    if tipo.formato[x] == 'object':
      categorica.append(tipo.feature[x])
      
  num = df[numerica]
  cat = df[categorica]
  
  return num, cat



In [9]:
train, relatorio_miss = DropaMiss(train, 80)

In [10]:
relatorio_miss
#Daqui vemos que as colunas dropadas foram ['PoolQC', 'MiscFeature', 'Alley', 'Fence']

,index,P_miss
71,PoolQC,99.520548
73,MiscFeature,96.301370
5,Alley,93.767123
72,Fence,80.753425
56,FireplaceQu,47.260274
...,...,...
38,Heating,0.000000
1,MSZoning,0.000000
40,CentralAir,0.000000
42,1stFlrSF,0.000000


In [11]:
#vamos dropar no df test as mesmas colunas que dropamos no df train
test = test.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1)

In [12]:
#dividindo o df train em dois df: numerica e categorica
numerica_train, categorica_train = AutoFrames(train)

In [13]:
#dividindo o df test em dois df: numerica e categorica de modo que tenha
#as mesmas colunas dos novos df de train
numerica_test = test[numerica_train.columns]
categorica_test = test[categorica_train.columns]

In [14]:
#vamos começar tratando a base numérica

In [15]:
numerica_train

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,6,5,1999,2000,0.0,0,0,953,953,953,694,0,1647,0,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,8,2007
1456,85.0,13175,6,6,1978,1988,119.0,790,163,589,1542,2073,0,0,2073,1,0,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,2,2010
1457,66.0,9042,7,9,1941,2006,0.0,275,0,877,1152,1188,1152,0,2340,0,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,5,2010
1458,68.0,9717,5,6,1950,1996,0.0,49,1029,0,1078,1078,0,0,1078,1,0,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,4,2010


In [16]:
#Vamos dropar as colunas do df numerica que apresentarem o valor 0 como 90% ou mais dos casos
#90% de 1460 = 1314
lista_drop_numerica = ['LowQualFinSF','BsmtHalfBath', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

numerica_train = numerica_train.drop(lista_drop_numerica, axis = 1)
numerica_test = numerica_test.drop(lista_drop_numerica, axis = 1)

In [17]:
#vamos tratar os missings do df numerica do treino
numerica_train.isna().sum()

LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
GrLivArea          0
BsmtFullBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
MoSold             0
YrSold             0
dtype: int64

In [18]:
valores_train = {'LotFrontage':numerica_train['LotFrontage'].mean(), 'MasVnrArea':numerica_train['MasVnrArea'].mean(), 'GarageYrBlt':numerica_train['GarageYrBlt'].mean()}
numerica_train = numerica_train.fillna(value=valores_train)

In [19]:
#vamos tratar os missings do df numerica do test
numerica_test.isna().sum()

LotFrontage      227
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea        15
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
1stFlrSF           0
2ndFlrSF           0
GrLivArea          0
BsmtFullBath       2
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       78
GarageCars         1
GarageArea         1
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
MoSold             0
YrSold             0
dtype: int64

In [20]:
valores_test = {'LotFrontage':numerica_train['LotFrontage'].mean(), 'MasVnrArea':numerica_train['MasVnrArea'].mean(), 'BsmtFinSF1':numerica_train['BsmtFinSF1'].mean(), 'BsmtFinSF2':numerica_train['BsmtFinSF2'].mean(),
                'BsmtUnfSF':numerica_train['BsmtUnfSF'].mean(), 'TotalBsmtSF':numerica_train['TotalBsmtSF'].mean(), 'BsmtFullBath':numerica_train['BsmtFullBath'].mean(), 'GarageYrBlt':numerica_train['GarageYrBlt'].mean(),
                'GarageCars':numerica_train['GarageCars'].mean(), 'GarageArea':numerica_train['GarageArea'].mean()}
numerica_test = numerica_test.fillna(value=valores_test)

In [21]:
#agora vamos tratar a base categórica

In [22]:
categorica_train

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60A,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Attchd,RFn,TA,TA,Y,WD,Normal
1,20A,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,60A,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,70A,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,60A,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60A,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
1456,20A,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,TA,TA,CBlock,Gd,TA,No,ALQ,Rec,GasA,TA,Y,SBrkr,TA,Min1,TA,Attchd,Unf,TA,TA,Y,WD,Normal
1457,70A,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,None,Ex,Gd,Stone,TA,Gd,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
1458,20A,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,TA,TA,Mn,GLQ,Rec,GasA,Gd,Y,FuseA,Gd,Typ,NaN,Attchd,Unf,TA,TA,Y,WD,Normal


In [23]:
#get_dummies
categorica_train = pd.get_dummies(categorica_train, drop_first=False, dtype=None)
categorica_test = pd.get_dummies(categorica_test, drop_first=False, dtype=None)

In [24]:
#criando listas para comparar os df train e test
lista_colunas_train = categorica_train.columns.to_list()
lista_colunas_test = categorica_test.columns.to_list()

In [25]:
#comparando (vendo o que está no train e não está no test)
Z = set(lista_colunas_train) - set(lista_colunas_test)
Z

{'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'Electrical_Mix',
 'Exterior1st_ImStucc',
 'Exterior1st_Stone',
 'Exterior2nd_Other',
 'GarageQual_Ex',
 'Heating_Floor',
 'Heating_OthW',
 'HouseStyle_2.5Fin',
 'RoofMatl_ClyTile',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll',
 'Utilities_NoSeWa'}

In [26]:
#criando colunas no df test que tinham no train, mas não no test 
for col in Z:
  categorica_test[col] = 0

In [27]:
#atualizando
lista_colunas_train = categorica_train.columns.to_list()
lista_colunas_test = categorica_test.columns.to_list()

In [28]:
#comparando (vendo o que está no test e não está no train)
Z = set(lista_colunas_test) - set(lista_colunas_train)
Z

{'MSSubClass_150A'}

In [29]:
#dropando do test o que está no test e não está no train
categorica_test = categorica_test.drop('MSSubClass_150A', axis = 1)

In [30]:
#atualizando
lista_colunas_train = categorica_train.columns.to_list()
lista_colunas_test = categorica_test.columns.to_list()

In [31]:
#verificando se agora as colunas estão iguais
set(lista_colunas_test) == set(lista_colunas_train)

True

In [32]:
#Fazendo o merge dos dataframes
abt_Train = pd.merge(numerica_train, categorica_train, how ='left', on=None, left_on=None, right_on=None, left_index=True, right_index=True)
abt_Test = pd.merge(numerica_test, categorica_test, how ='left', on=None, left_on=None, right_on=None, left_index=True, right_index=True)

In [33]:
#normalização do df train
scaler = MinMaxScaler()
scaler.fit(abt_Train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [34]:
abt_train = pd.DataFrame(scaler.transform(abt_Train), columns=abt_Train.columns)

In [35]:
#normalização do df test
scaler2 = MinMaxScaler()
scaler2.fit(abt_Test)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [36]:
abt_test = pd.DataFrame(scaler2.transform(abt_Test), columns=abt_Test.columns)

In [37]:
#Criando e rodando o modelo

In [38]:
desenvolvimento = abt_train.copy()
X_test = abt_test.copy()

In [39]:
#dividindo nosso train em desenvolvimento e validação
X_desenv, X_valid, y_desenv, y_valid = train_test_split(desenvolvimento, respostas, test_size = 0.7)

In [40]:
#vamos usar o algoritmo RandomForestRegressor() e vamos usar
#o GridSearchCV para buscar bons valores para 'max_depth' e 'n_estimators'
GS_LGBM = GridSearchCV(estimator = RandomForestRegressor(), param_grid={'max_depth' : [8, 16, 24], 'n_estimators' : [1000, 1400, 1800]})

In [41]:
GS_LGBM.fit(X_desenv, y_desenv)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n

In [42]:
model = GS_LGBM.best_estimator_

In [43]:
model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=16, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [44]:
model.fit(X_desenv, y_desenv)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=16, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [46]:
erro = mean_squared_error(y_valid, model.predict(X_valid))
np.sqrt(erro)

33965.352769993784

In [47]:
previsao = model.predict(X_test)

In [48]:
#gerando csv resposta
final = pd.read_csv('test.csv')
final['SalePrice'] = previsao
final = final[['Id', 'SalePrice']]
final.to_csv('DesafioHousePriceF.csv', index = False, sep = ',')